# 신한은행 FAQ 데이타를 이용하여 스키마 정의 후 Vector Store 생성 

이 노트북은 신한은행 FAQ 데이타를 이용하여 오픈 서치 벡터 스토어에 데이터를 저장하고, 렉시컬, 시멘틱, 하이브리드 검색을 해보는 노트북 입니다.

---

## [중요] 사전 실행 노트북
이 노트북은 아래 두개의 셋업 노트북이 먼저 실행이 되어야 합니다.
- (1) Setup 노트북
    - 경로는 aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/00_setup/setup.ipynb 와 같습니다.
    -  [Setup Notebook](https://github.com/aws-samples/aws-ai-ml-workshop-kr/blob/master/genai/aws-gen-ai-kr/00_setup/setup.ipynb)
- (2) Amazon OpenSearch 설치 노트북    
    - 경로는 aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/00_setup/setup_opensearch.ipynb 와 같습니다.
    - [Setup OpenSearch](https://github.com/aws-samples/aws-ai-ml-workshop-kr/blob/master/genai/aws-gen-ai-kr/00_setup/setup_opensearch.ipynb)

## Setting
 - Auto Reload
 - path for utils

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os

def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)

module_path = ".."
add_python_path(module_path)
module_path = "../../.."
add_python_path(module_path)

python path: /home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot is added
sys.path:  ['/home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot/01_preprocess_docs', '/home/ec2-user/anaconda3/envs/python3/lib/python310.zip', '/home/ec2-user/anaconda3/envs/python3/lib/python3.10', '/home/ec2-user/anaconda3/envs/python3/lib/python3.10/lib-dynload', '', '/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages', '/home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot']
python path: /home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr is added
sys.path:  ['/home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot/01_preprocess_docs', '/home/ec2-user/anaconda3/envs/python3/lib/python310.zip', '/home/ec2-user/anaconda3/envs/python3/lib/python3.10', '/home/ec2-user/anaconda3/envs/python3/lib/python3.10/lib-

## 1. Bedrock Client 생성

In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [4]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Claude-V3-Haiku': 'anthropic.claude-3-haiku-20240307-v1:0',
 'Claude-V3-Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


## 2. Embedding 모델 로딩

## Embedding Model 선택

In [5]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain_community.chat_models import BedrockChat

In [6]:
llm_emb = BedrockEmbeddings(
    client=boto3_bedrock,
    model_id=bedrock_info.get_model_id(
        model_name="Titan-Embeddings-G1"
    )
)
dimension = 1536
llm_emb

BedrockEmbeddings(client=<botocore.client.BedrockRuntime object at 0x7f602dae91e0>, region_name=None, credentials_profile_name=None, model_id='amazon.titan-embed-text-v1', model_kwargs=None, endpoint_url=None, normalize=False)

## 3.데이터 준비 

###  신한은행 FAQ 데이터 세트로 구현
- [중요] 저자 및 동료가 아래의 웹사이트에서 크로링한 기준으로 구성 하였습니다.
- 인터넷뱅킹 FAQ > 스마트뱅킹 No.1 ~ N. 89 로 구성되었습니다. 
- https://www.shinhan.com/hpe/index.jsp#050101020000

In [7]:
import pandas as pd

In [8]:
data_file_path = "./data/shinhan/fsi_smart_faq_ko.csv"
df = pd.read_csv(data_file_path)
df

,no,Category,Information,type,Source
0,91,아마존 은행의 타기관OTP 이용등록방법 알려주세요,아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 ...,인터넷뱅킹,아마존은행
1,90,아마존 공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,아마존은행
2,88,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행
3,89,타기관OTP 이용등록방법 알려주세요,타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다....,인터넷뱅킹,신한은행
4,88,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행
...,...,...,...,...,...
87,5,인터넷에서 이체한 거래의 상세내역 및 영수증을 인쇄하고 싶어요,"인터넷상에서 이체한 거래의 경우 인터넷상에서 제공하는 영수증은 없으며, 거래의 참고...",인터넷뱅킹,신한은행
88,4,타행으로 자동이체를 신청하고 싶은데요,인터넷뱅킹을 통한 타은행 계좌로의 자동이체(납부자 자동이체)신청이 가능하며 수수료는...,인터넷뱅킹,신한은행
89,3,공과금 자동이체 신청이 가능한가요?,"인터넷뱅킹에 로그인하신 후 ""공과금/법원 > 공과금센터"" 페이지에 가시면 ""지로자동...",인터넷뱅킹,신한은행
90,2,인터넷뱅킹 거래에 대한 책임 범위를 알고싶습니다.,기본적으로 은행이 인터넷뱅킹 사고에 대한 책임을 집니다. 해커의 침입에 의해 자금이...,인터넷뱅킹,신한은행


### 데이터 전처리
- 여기서 no 는 제거 합니다. 

In [9]:
os.makedirs("data", exist_ok=True)

In [10]:
def preprocess_data(df):

    ldf = df.copy()
    ldf.rename(columns={'Category': 'ask'}, inplace=True)
    df_index = ldf.drop(['no'], axis=1)
    df_index.to_csv("./data/shinhan/fsi_smart_faq_ko_processed.csv", index=None)

    return df_index

pre_df = preprocess_data(df)
pre_df.head(5)

,ask,Information,type,Source
0,아마존 은행의 타기관OTP 이용등록방법 알려주세요,아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 ...,인터넷뱅킹,아마존은행
1,아마존 공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,아마존은행
2,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행
3,타기관OTP 이용등록방법 알려주세요,타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다....,인터넷뱅킹,신한은행
4,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행


### CSVLoader 로 문서 로딩

In [11]:
import time
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter, SpacyTextSplitter

In [12]:
loader = CSVLoader(
    file_path="./data/shinhan/fsi_smart_faq_ko_processed.csv",
    csv_args={
        "delimiter": ",",
    },
    source_column="Source",
    encoding="utf-8"
)

documents_fsi = loader.load()

In [13]:
documents_fsi[0]

Document(page_content='ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요\nInformation: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \r\n[경로]\r\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \r\n- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\r\n \r\n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\r\n\r\n기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.\ntype: 인터넷뱅킹\nSource: 아마존은행', metadata={'source': '아마존은행', 'row': 0})

In [14]:
def create_metadata(docs):
    # # add a custom metadata field, such as timestamp
    for idx, doc in enumerate(docs):

        print ("previous:", doc.metadata)
        # type 을 메타 데이타로 저장
        stype = doc.page_content.split("type: ")[1].split("\n")[0]
        split_content = doc.page_content.split("type: ")
        content = split_content[0]
        metadata = split_content[1]
        doc.metadata['type'] = metadata.split("\n")[0]        
        doc.page_content = content # metadata 제외하고 content 만 저장
        doc.metadata['timestamp'] = time.time()

        print ("new:", doc.metadata)
        print ("=====")

create_metadata(documents_fsi)

previous: {'source': '아마존은행', 'row': 0}
new: {'source': '아마존은행', 'row': 0, 'type': '인터넷뱅킹', 'timestamp': 1711082791.233616}
=====
previous: {'source': '아마존은행', 'row': 1}
new: {'source': '아마존은행', 'row': 1, 'type': '인증서', 'timestamp': 1711082791.233657}
=====
previous: {'source': '신한은행', 'row': 2}
new: {'source': '신한은행', 'row': 2, 'type': '인증서', 'timestamp': 1711082791.233691}
=====
previous: {'source': '신한은행', 'row': 3}
new: {'source': '신한은행', 'row': 3, 'type': '인터넷뱅킹', 'timestamp': 1711082791.2337236}
=====
previous: {'source': '신한은행', 'row': 4}
new: {'source': '신한은행', 'row': 4, 'type': '인증서', 'timestamp': 1711082791.2337554}
=====
previous: {'source': '신한은행', 'row': 5}
new: {'source': '신한은행', 'row': 5, 'type': '금융인증서', 'timestamp': 1711082791.2337868}
=====
previous: {'source': '신한은행', 'row': 6}
new: {'source': '신한은행', 'row': 6, 'type': '금융인증서', 'timestamp': 1711082791.2338178}
=====
previous: {'source': '신한은행', 'row': 7}
new: {'source': '신한은행', 'row': 7, 'type': '금융인증서', 'timestamp':

In [15]:
print (len(documents_fsi))
print (documents_fsi[0])

92
page_content='ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요\nInformation: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \r\n[경로]\r\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \r\n- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\r\n \r\n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\r\n\r\n기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.\n' metadata={'source': '아마존은행', 'row': 0, 'type': '인터넷뱅킹', 'timestamp': 1711082791.233616}


### Text Spliter 로 청킹
참고: 검색된 문서/텍스트는 질문에 대답하기에 충분한 정보를 포함할 만큼 커야 합니다. 하지만 LLM 프롬프트에 들어갈 만큼 충분히 작습니다. <BR>
또한 임베딩 모델에는 입력 토큰 길이는 KoSimCSERobert는 512개, titanEmbedding(8,912개) 토큰으로 제한되어 있습니다. <BR>
이 사용 사례를 위해 [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html)를 사용하여 500자가 겹치는 약 92자의 청크를 생성합니다.

In [16]:
chunk_size = 2048
chunk_overlap = 50

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=["\n\n", "\n", ".", " ", ""],
    length_function=len,
)

chunk_docs = text_splitter.split_documents(documents_fsi)
print(f"Number of documents after split and chunking={len(chunk_docs)}")

Number of documents after split and chunking=92


In [17]:
chunk_docs[0]

Document(page_content='ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요\nInformation: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \r\n[경로]\r\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \r\n- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\r\n \r\n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\r\n\r\n기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.', metadata={'source': '아마존은행', 'row': 0, 'type': '인터넷뱅킹', 'timestamp': 1711082791.233616})

## 4. Index 생성

### Index 이름 결정

In [18]:
# index_name = <your index>
index_name = "v1-faq-shinhan-bank"

#### Save reranker endpoint to Parameter Store

In [19]:
import boto3
from local_utils.ssm import parameter_store

In [20]:
region=boto3.Session().region_name
pm = parameter_store(region)

In [21]:
pm.put_params(
    key="opensearch_index_name",
    value=f'{index_name}',
    overwrite=True,
    enc=False
)

Parameter stored successfully.


## Index 스키마 정의

In [22]:
index_body = {
    'settings': {
        'analysis': {
            'analyzer': {
                'my_analyzer': {
                         'char_filter':['html_strip'],
                    'tokenizer': 'nori',
                    'filter': [
                        #'nori_number',
                        #'lowercase',
                        #'trim',
                        'my_nori_part_of_speech'
                    ],
                    'type': 'custom'
                }
            },
            'tokenizer': {
                'nori': {
                    'decompound_mode': 'mixed',
                    'discard_punctuation': 'true',
                    'type': 'nori_tokenizer'
                }
            },
            "filter": {
                "my_nori_part_of_speech": {
                    "type": "nori_part_of_speech",
                    "stoptags": [
                        "J", "XSV", "E", "IC","MAJ","NNB",
                        "SP", "SSC", "SSO",
                        "SC","SE","XSN","XSV",
                        "UNA","NA","VCP","VSV",
                        "VX"
                    ]
                }
            }
        },
        'index': {
            'knn': True,
            'knn.space_type': 'cosinesimil'  # Example space type
        }
    },
    'mappings': {
        'properties': {
            'metadata': {
                'properties': {
                    'source': {'type': 'keyword'},
                    'row': {'type': 'long'},
                    'type': {'type': 'keyword'},
                    'timestamp': {'type': 'float'},
                }
            },
            'text': {
                'analyzer': 'my_analyzer',
                'search_analyzer': 'my_analyzer',
                'type': 'text'
            },
            'vector_field': {
                'type': 'knn_vector',
                'dimension': f"{dimension}" # Replace with your vector dimension
            }
        }
    }
}


## 5. LangChain OpenSearch VectorStore 생성 
### 선수 조건


#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.

In [23]:
import boto3
from utils.ssm import parameter_store

In [24]:
region=boto3.Session().region_name
pm = parameter_store(region)

opensearch_domain_endpoint = pm.get_params(
    key="opensearch_domain_endpoint",
    enc=False
)

opensearch_user_id = pm.get_params(
    key="opensearch_user_id",
    enc=False
)

opensearch_user_password = pm.get_params(
    key="opensearch_user_password",
    enc=True
)


In [25]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

### OpenSearch Client 생성

In [26]:
from local_utils.opensearch import opensearch_utils

In [27]:
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

### 오픈 서치 인덱스 생성 
- 오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [28]:
from local_utils.opensearch import opensearch_utils

In [29]:
index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    index_name
)

if index_exists:
    opensearch_utils.delete_index(
        os_client,
        index_name
    )

opensearch_utils.create_index(os_client, index_name, index_body)
index_info = os_client.indices.get(index=index_name)
print("Index is created")
pprint(index_info)

index_name=v1-faq-shinhan-bank, exists=True

Deleting index:
{'acknowledged': True}

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'v1-faq-shinhan-bank'}
Index is created
{'v1-faq-shinhan-bank': {'aliases': {},
                         'mappings': {'properties': {'metadata': {'properties': {'row': {'type': 'long'},
                                                                                 'source': {'type': 'keyword'},
                                                                                 'timestamp': {'type': 'float'},
                                                                                 'type': {'type': 'keyword'}}},
                                                     'text': {'analyzer': 'my_analyzer',
                                                              'type': 'text'},
                                                     'vector_field': {'dimension': 1536,
                                                          

### 랭체인 인덱스 연결 오브젝트 생성

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

In [30]:
from langchain.vectorstores import OpenSearchVectorSearch

In [31]:
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss=False,
    engine="faiss",
    space_type="l2",
    bulk_size=100000,
    timeout=60
)
vector_db

### OpenSearch 에 문서 삽입

In [32]:
%%time

vector_db.add_documents(
    documents = chunk_docs, 
    vector_field = "vector_field",
    bulk_size = 1000000
)


CPU times: user 296 ms, sys: 8.23 ms, total: 304 ms
Wall time: 25.5 s


['1423af06-5321-465a-a5fb-bfffb8d3fdce',
 '525d2bb3-21bd-4904-ba38-93c1f2ee23f5',
 '3d80352c-2a73-4b51-9c0d-97b63bc0afe6',
 '6f63bf8f-3b87-422a-8904-4d524a7dc02f',
 '19cb2e3e-40fa-4606-be65-9504d8b690b1',
 '09ccfb5f-aa85-4da7-bac8-beff577fd6e1',
 '3f4888ca-8ea6-4817-9c82-377bb64eda94',
 '09ce1a72-08f2-47d1-914e-e13ef51f76be',
 'ae0b3457-1a86-4bdd-ab3e-eb45dc0791c1',
 '6a1e6320-f555-4ef8-9bc5-93cce1c684a0',
 '1b645d4a-230d-470b-805d-fe875f6bad6b',
 'b18a3ab8-adc2-47ab-b1df-9e19f836fa2a',
 'bbd76fcc-2c1e-46c9-841e-e9d7cb15f9ad',
 'fa1f5638-746e-40f5-a013-a75ab4343eef',
 '7510f38c-14a8-404c-9012-2f2e625c2b0a',
 '23cb12d5-4409-41a1-98c2-3d065770d7f2',
 'a703f979-9b53-44e2-a84b-f373cc2f5c06',
 '8531c5b5-853d-4718-8dfb-ea6ba5cb8656',
 '0d60986a-718e-449b-bf44-83b1a89a2d45',
 '43e494e6-595d-42e6-ba60-6d00a134ad4b',
 '99a5c3c1-51d7-4a30-a5a5-c0d7cb2b4318',
 '79b66e45-c75a-427a-981f-885ee2fd84ac',
 'fd3460af-699b-4c6f-97ea-260d0af9c277',
 'e7ffc7e6-e8b8-41ea-bead-3a511679281d',
 'b5ba4d62-32f4-

## 6. 검색 및 질의 응답 테스트

In [33]:
from utils.rag import retriever_utils
from utils.rag import show_context_used
from langchain.schema.output_parser import StrOutputParser
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

### LLM 선택

In [34]:
llm_text = BedrockChat(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-Haiku"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens": 1024,
        "temperature":0,
        "top_p":0.9,
        "stop_sequences": ["\n\nHuman"],
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)
llm_text

BedrockChat(client=<botocore.client.BedrockRuntime object at 0x7f602dae91e0>, model_id='anthropic.claude-3-haiku-20240307-v1:0', model_kwargs={'max_tokens': 1024, 'temperature': 0, 'top_p': 0.9, 'stop_sequences': ['\n\nHuman']}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f6049d7ae00>])

### QA prompt

In [35]:
system_prompt = '''
                You are a master answer bot designed to answer user's questions.
                I'm going to give you contexts which consist of texts, tables and images.
                Read the contexts carefully, because I'm going to ask you a question about it.
                '''
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)

human_prompt = [
    {
        "type": "text",
        "text": '''
                Here is the contexts as texts: <contexts>{contexts}</contexts>

                Only using the context as above, answer the following question with the rules as below:
                    - Don't insert XML tag such as <contexts> and </contexts> when answering.
                    - Write as much as you can
                    - Be courteous and polite
                    - Only answer the question if you can find the answer in the contexts with certainty.

                Question:
                {question}

                If the answer is not in the contexts, just say "주어진 내용에서 관련 답변을 찾을 수 없습니다."

                '''
    }
]
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

prompt = ChatPromptTemplate.from_messages(
    [system_message_template, human_message_template]
)


### QA chain

In [36]:
chain = prompt | llm_text | StrOutputParser()

### Semantic 검색

In [37]:
query = "중지된 경우 이체"

search_filter=[
    #{"term": {"metadata.source": "신한은행"}},
    #{"term": {"metadata.type": "인터넷뱅킹"}},
]

In [38]:
similar_docs_semantic = retriever_utils.get_semantic_similar_docs(
    index_name=index_name,
    os_client=os_client,
    llm_emb=llm_emb,
    query=query,
    k=5,
    boolean_filter=search_filter,
    hybrid=False
)
show_context_used(similar_docs_semantic)


-----------------------------------------------
1. Chunk: 402 Characters
-----------------------------------------------
ask: 이체한도란 무엇인가요?
Information: 이체한도란 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등을 통하여 이체할 수 있는 거래한도를 말합니다. ① 1일/1회 이체한도 1일 이체한도란 하루에 거래할 수 있는
이체금액의 합을 말하여, 1회 이체한도란 1회에 이체하실 수 있는 금액의 한도를 말합니다. ② 통합이체한도 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등을 고객님이 사용하시는 경우 고객님이
1일 또는 1회에 이체하실 수 있는 이체한도를 말합니다. 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등에서 고객님이 거래하신 이체금액의 합이 통합이체한도의 범위를 초과할 수 없습니다. ③
최고이체한도 금융감독원에서는 개인의 전자금융 시 이용할 수 있는 이체한도의 가이드라인을 정하고 있습니다.(인터넷뱅킹 : 1일 5억원, 1회 1억원 이내)
metadata:
 {'source': '신한은행', 'row': 39, 'type': '인터넷뱅킹', 'timestamp': 1711082791.2348623, 'id':
'b3312a85-44d7-44ec-aa68-628e101f4b11'}

-----------------------------------------------
2. Chunk: 584 Characters
-----------------------------------------------
ask: 이체한도 증액은 어떻게 하나요?
Information: 영업점 방문하지 않아도 모바일뱅킹 신한 쏠(SOL) 에서 비대면실명 인증후 통합이체한도 증액 가능합니다. 통합이체한도는 이용하시는 보안매체에 따라,
보안등급별 최고 한도내에서 증액가능합니다.
OTP카드 창구 발급 시 5,000원(카드형 10,000원)의 수수료가 부과됩니다. 당행에서만 사용가능한 모바일OTP는 영업점 방문

In [39]:
answer= chain.invoke(
    {
        "contexts": similar_docs_semantic,
        "question": query
    }
)

print("\n##############################")
print("query: \n", query)
print("answer: \n", answer)

주어진 내용에서 중지된 경우 이체에 대한 정보를 찾을 수 없습니다. 이 문서들은 이체한도, 이체한도 증액, 온라인 서비스 신청 시 구비서류, 인터넷 이체 거래 내역 조회 및 인쇄 방법 등에 대한 정보를 제공하고 있지만, 중지된 경우 이체에 대한 내용은 포함되어 있지 않습니다.
##############################
query: 
 중지된 경우 이체
answer: 
 주어진 내용에서 중지된 경우 이체에 대한 정보를 찾을 수 없습니다. 이 문서들은 이체한도, 이체한도 증액, 온라인 서비스 신청 시 구비서류, 인터넷 이체 거래 내역 조회 및 인쇄 방법 등에 대한 정보를 제공하고 있지만, 중지된 경우 이체에 대한 내용은 포함되어 있지 않습니다.


## Lexical 검색

In [40]:
query = "중지된 경우 이체"

search_filter=[
    #{"term": {"metadata.source": "신한은행"}},
    #{"term": {"metadata.type": "인터넷뱅킹"}},
]

In [41]:
similar_docs_lexical = retriever_utils.get_lexical_similar_docs(
    index_name=index_name,
    os_client=os_client,
    query=query,
    k=5,
    filter=search_filter
)
show_context_used(similar_docs_lexical)


-----------------------------------------------
1. Chunk: 591 Characters
-----------------------------------------------
ask: 12개월 장기미이체로 이체서비스가 중지된 경우 어떻게 해제하나요?
Information: 최근 12개월동안 인터넷뱅킹/폰뱅킹/모바일뱅킹에서 이체서비스를 이용하지 않은 경우 고객님의 금융자산 보호를 위하여 이체서비스가 중단됩니다. 다시 이체를
원하시는 경우에는 인터넷뱅킹/폰뱅킹/신한쏠(SOL)에서 이체정지 해제를 하실 수 있습니다.(단, 인터넷뱅킹 출금계좌가 등록되어 있는 경우) [경로] ① 당행 고객정보에 등록되어있는
연락처로 ARS 추가인증 가능한 경우: - 인터넷뱅킹 로그인→ 사용자관리→ 인터넷뱅킹관리→ 장기미사용 정지해제 (또는) - 쏠(SOL) 로그인→ 전체메뉴→ 이체 → 이체관리→
장기미사용정지 해제 ② ARS 추가인증이 불가한 경우 (해외체류 고객등) : - 인터넷뱅킹에서는 해제 어려우며, 쏠 (SOL)에서 해제 가능합니다. 쏠(SOL)의 경로는 위와
동일하며, 비대면 실명인증후 해제 가능합니다. 비대면실명인증 - 만 14세이상 가능 - 신분증확인 (주민등록증 또는 운전면허증 또는 여권) - 본인확인 (본인명의 계좌인증 또는
영상통화) 진행 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다
metadata:
 {'source': '신한은행', 'row': 40, 'type': '인터넷뱅킹', 'timestamp': 1711082791.2348938, 'id':
'd1d16682-c69b-48e2-a79b-f67d517e0d51'}

-----------------------------------------------
2. Chunk: 458 Characters
-----------------------------------------------
ask: 인터넷에서 이체한 거래의 상세내역 및 영

In [42]:
answer = chain.invoke(
    {
        "contexts": similar_docs_lexical,
        "question": query
    }
)

print("\n##############################")
print("query: \n", query)
print("answer: \n", answer)

주어진 내용에 따르면, 최근 12개월 동안 인터넷뱅킹/폰뱅킹/모바일뱅킹에서 이체서비스를 이용하지 않은 경우 고객의 금융자산 보호를 위해 이체서비스가 중단됩니다. 다시 이체를 원하시는 경우에는 인터넷뱅킹/폰뱅킹/신한쏠(SOL)에서 이체정지 해제를 할 수 있습니다. 

ARS 추가인증이 가능한 경우 인터넷뱅킹이나 신한쏠(SOL)에서 직접 장기미사용 정지를 해제할 수 있습니다. ARS 추가인증이 불가능한 경우(해외체류 고객 등)에는 신한쏠(SOL)에서 비대면 실명인증 후 정지를 해제할 수 있습니다.

기타 궁금한 사항은 신한은행 고객센터 1599-8000으로 문의하시기 바랍니다.
##############################
query: 
 중지된 경우 이체
answer: 
 주어진 내용에 따르면, 최근 12개월 동안 인터넷뱅킹/폰뱅킹/모바일뱅킹에서 이체서비스를 이용하지 않은 경우 고객의 금융자산 보호를 위해 이체서비스가 중단됩니다. 다시 이체를 원하시는 경우에는 인터넷뱅킹/폰뱅킹/신한쏠(SOL)에서 이체정지 해제를 할 수 있습니다. 

ARS 추가인증이 가능한 경우 인터넷뱅킹이나 신한쏠(SOL)에서 직접 장기미사용 정지를 해제할 수 있습니다. ARS 추가인증이 불가능한 경우(해외체류 고객 등)에는 신한쏠(SOL)에서 비대면 실명인증 후 정지를 해제할 수 있습니다.

기타 궁금한 사항은 신한은행 고객센터 1599-8000으로 문의하시기 바랍니다.


## 하이브리드 검색

In [43]:
query = "중지된 경우 이체"

search_filter=[
    {"term": {"metadata.source": "신한은행"}},
    {"term": {"metadata.type": "인터넷뱅킹"}},
]

In [44]:
%%time
similar_docs_hybrid = retriever_utils.search_hybrid(
    query=query,
    k=7,
    index_name=index_name,
    os_client=os_client,
    filter=search_filter,
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"]
    ensemble_weights=[0.49, 0.51], # semantic, lexical
    async_mode=True,
    llm_emb=llm_emb,
    verbose=True
)

##############################
async_mode
##############################
True
##############################
reranker
##############################
False
##############################
rag_fusion
##############################
False
##############################
HyDE
##############################
False
##############################
parent_document
##############################
False
##############################
complex_document
##############################
False
##############################
similar_docs_semantic
##############################

Score: 1.0
['ask: 이체한도란 무엇인가요?', 'Information: 이체한도란 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등을 통하여 이체할 수 있는 거래한도를 말합니다. ① 1일/1회 이체한도 1일 이체한도란 하루에 거래할 수 있는 이체금액의 합을 말하여, 1회 이체한도란 1회에 이체하실 수 있는 금액의 한도를 말합니다. ② 통합이체한도 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등을 고객님이 사용하시는 경우 고객님이 1일 또는 1회에 이체하실 수 있는 이체한도를 말합니다. 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등에서 고객님이 거래하신 이체금액의 합이 통합이체한도의 범위를 초과할 수 없습니다. ③ 최고이체한도 금융감독원에서는 개인의 전자금융 시 이용할 수 있는 이체한도의 가이드라인을 정하고 있습니다.(인터넷뱅킹 : 1일 5억원, 1회 1억원 이내)']
{'source': '신한은행',

In [45]:
answer = chain.invoke(
    {
        "contexts": similar_docs_hybrid,
        "question": query
    }
)

print("\n##############################")
print("query: \n", query)
print("answer: \n", answer)

주어진 내용에 따르면, 최근 12개월 동안 인터넷뱅킹/폰뱅킹/모바일뱅킹에서 이체서비스를 이용하지 않은 경우 고객님의 금융자산 보호를 위해 이체서비스가 중단됩니다. 다시 이체를 원하시는 경우에는 인터넷뱅킹/폰뱅킹/신한쏠(SOL)에서 이체정지 해제를 하실 수 있습니다. 

ARS 추가인증이 가능한 경우에는 인터넷뱅킹이나 신한쏠(SOL)에서 직접 장기미사용 정지를 해제할 수 있습니다. ARS 추가인증이 불가능한 경우(해외체류 고객 등)에는 신한쏠(SOL)에서 비대면 실명인증 후 장기미사용 정지를 해제할 수 있습니다.

기타 궁금한 사항은 신한은행 고객센터 1599-8000으로 문의하시기 바랍니다.
##############################
query: 
 중지된 경우 이체
answer: 
 주어진 내용에 따르면, 최근 12개월 동안 인터넷뱅킹/폰뱅킹/모바일뱅킹에서 이체서비스를 이용하지 않은 경우 고객님의 금융자산 보호를 위해 이체서비스가 중단됩니다. 다시 이체를 원하시는 경우에는 인터넷뱅킹/폰뱅킹/신한쏠(SOL)에서 이체정지 해제를 하실 수 있습니다. 

ARS 추가인증이 가능한 경우에는 인터넷뱅킹이나 신한쏠(SOL)에서 직접 장기미사용 정지를 해제할 수 있습니다. ARS 추가인증이 불가능한 경우(해외체류 고객 등)에는 신한쏠(SOL)에서 비대면 실명인증 후 장기미사용 정지를 해제할 수 있습니다.

기타 궁금한 사항은 신한은행 고객센터 1599-8000으로 문의하시기 바랍니다.


# A. Reference

- [Building a RAG AI with OpenSearch Serverless and LangChain](https://caylent.com/blog/building-a-rag-with-open-search-serverless-and-lang-chain)